In [1]:
import os
import math
import random
import zipfile
import collections

import tensorflow as tf
import numpy as np

from six.moves import urllib

In [2]:
BASE_DIR = '.'
DATA_DIR = os.path.join(BASE_DIR, 'data/')

## Word2vec

This tutorial goes through a model which convert words into vectors called Word2vec.

Word2vec is a group of related models that are used to produce word embeddings.Word2vec takes as its input a large corpus of text and produces a vector space, typically of several hundred dimensions, with each unique word in the corpus being assigned a corresponding vector in the space.

Reference:

[Tensorflow: Vector Representations of Words](https://www.tensorflow.org/tutorials/word2vec)

[Wikipedia: Word2vec](https://en.wikipedia.org/wiki/Word2vec)

## Step 1: Download and read data

In [3]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
    """Download a file if not present"""
    if not os.path.exists(DATA_DIR+filename):
        filename, _ = urllib.request.urlretrieve(url+filename, DATA_DIR+filename)
    statinfo = os.stat(DATA_DIR+filename)
    if statinfo.st_size == expected_bytes:
        print('Found and verified', filename)
    else:
        print(statinfo.st_size)
        raise Exception(
            'Failed to verify ' + filename + '. Can you get to it with a browser?')
    return filename

In [4]:
filename = maybe_download('text8.zip', 31344016)

('Found and verified', 'text8.zip')


In [5]:
def read_data(filename):
    with zipfile.ZipFile(DATA_DIR+filename) as f:
        data = f.read(f.namelist()[0]).split()
    return data

In [6]:
words = read_data(filename)
print('Data size', len(words))

('Data size', 17005207)


## Step 2: Build dictionary and replace rare words by UNK token

In [7]:
vocabulary_size = 50000

def build_dataset(words, vocabulary_size):
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(vocabulary_size-1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    reverse_dictionary = dict((v,k) for k,v in dictionary.iteritems())
    data = list()
    unk_count = 0
    for word in words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0
            unk_count = 0
        data.append(index)
        count[0][1] = unk_count
    return data, count, dictionary, reverse_dictionary

In [8]:
data, count, dictionary, reverse_dictionary = build_dataset(words, vocabulary_size)

In [9]:
del words

In [10]:
print('Most common words (+UNK)', count[:5])
print('Sample data')
print(data[:8])
print([reverse_dictionary[i] for i in data[:8]])

('Most common words (+UNK)', [['UNK', 0], ('the', 1061396), ('of', 593677), ('and', 416629), ('one', 411764)])
Sample data
[5239, 3084, 12, 6, 195, 2, 3137, 46]
['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first']


## Step 3: Function to generate a training batch for the skip-gram model

In [11]:
def generate_batch(batch_size, num_skips, skip_window):
    global data_index
    assert num_skips <= 2 * skip_window
    assert batch_size % num_skips == 0
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1
    buffer = collections.deque(maxlen=span)
    for _ in range(span):
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    for i in range(batch_size // num_skips):
        target = skip_window
        targets_to_avoid = [skip_window]
        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0, span-1)
            targets_to_avoid.append(target)
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j] = buffer[target]
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    data_index = (data_index + len(data) - span) % len(data)
    return batch, labels

In [12]:
data_index = 0

batch, labels = generate_batch(batch_size=8, num_skips=2, skip_window=1)
for i in range(8):
    print(batch[i], reverse_dictionary[batch[i]], '->', labels[i, 0], reverse_dictionary[labels[i, 0]])

(3084, 'originated', '->', 5239, 'anarchism')
(3084, 'originated', '->', 12, 'as')
(12, 'as', '->', 3084, 'originated')
(12, 'as', '->', 6, 'a')
(6, 'a', '->', 12, 'as')
(6, 'a', '->', 195, 'term')
(195, 'term', '->', 6, 'a')
(195, 'term', '->', 2, 'of')


## Step 4: Skip-gram Model

In [13]:
batch_size = 128
embedding_size = 128
skip_window = 1
num_skips = 2

In [14]:
valid_size = 8
valid_window = 100
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
num_sampled = 64

In [15]:
graph = tf.Graph()

with graph.as_default():
    # Input data
    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
    
    with tf.device("/cpu:0"):
        # Vector Representation and its lookup
        embeddings = tf.Variable(
            tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0)
        )
        embed = tf.nn.embedding_lookup(embeddings, train_inputs)
        
        # weights and biases for NCE loss
        nce_weights = tf.Variable(
            tf.truncated_normal(
                [vocabulary_size, embedding_size], 
                stddev=1.0 / math.sqrt(embedding_size)
            )
        )
        nce_biases = tf.Variable(
            tf.zeros([vocabulary_size])
        )
        
        # NCE loss
        loss = tf.reduce_mean(
            tf.nn.nce_loss(
                weights=nce_weights,
                biases=nce_biases,
                labels=train_labels,
                inputs=embed,
                num_sampled=num_sampled,
                num_classes=vocabulary_size
            )
        )
        
        # Optimizer
        optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)
        
        # Cosine Similarity between valid examples and all embeddings
        norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
        normalized_embeddings = embeddings / norm
        valid_embeddings = tf.nn.embedding_lookup(
            normalized_embeddings, valid_dataset
        )
        similarity = tf.matmul(
            valid_embeddings, normalized_embeddings, transpose_b=True
        )
        
        """
        sess = tf.InteractiveSession()
        sess.run(tf.global_variables_initializer())
        print sess.run(valid_dataset)
        print [reverse_dictionary[i] for i in sess.run(valid_dataset)]
        print sess.run(valid_embeddings).shape
        print sess.run(normalized_embeddings).shape
        print sess.run(similarity).shape
        sess.close()
        """

## Step 5: Begin training

In [16]:
data_index = 0
num_steps = 100001

with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    print('Initialized')
    
    average_loss = 0
    for step in xrange(num_steps):
        batch_inputs, batch_labels = generate_batch(
            batch_size, num_skips, skip_window
        )
        feed_dict = {
            train_inputs: batch_inputs,
            train_labels: batch_labels
        }
        
        _, loss_val = sess.run([optimizer, loss], feed_dict=feed_dict)
        average_loss += loss_val
        
        if step % 2000 == 0:
            if step > 0:
                average_loss /= 2000
            print('Average loss at step %s: %s' % (step, average_loss))
            average_loss = 0
        
        if step % 10000 == 0:
            sim = similarity.eval()
            for i in xrange(valid_size):
                valid_word = reverse_dictionary[valid_examples[i]]
                top_k = 8
                nearest = (-sim[i, :]).argsort()[1:top_k + 1] # except self
                log_str = "Nearest to %s: " % valid_word
                for k in xrange(top_k):
                    close_word = reverse_dictionary[nearest[k]]
                    log_str = "%s %s," % (log_str, close_word)
                print(log_str)
    final_embeddings = normalized_embeddings.eval()

Initialized
Average loss at step 0: 282.45715332
Nearest to from:  relapse, politeness, tumors, preeminence, gimp, required, sarcasm, superficial,
Nearest to at:  wettest, gilmour, zimbabwean, rabbis, subtle, clements, ris, stocks,
Nearest to an:  deterrent, rabat, constitute, salesman, gillies, disturb, eruptions, arte,
Nearest to a:  lem, predominate, bauhaus, holliday, www, shareholders, kremlin, memorial,
Nearest to be:  deutschen, emphasises, carpenters, cfl, dp, manna, votes, folding,
Nearest to three:  adc, lexikon, comnena, alphonse, rockers, registries, interchangeability, immature,
Nearest to american:  bsod, illustrious, hrer, germania, postmodernist, hasty, willi, jerseys,
Nearest to during:  pompey, horst, trivia, burgos, mills, abbott, pear, sever,
Average loss at step 2000: 113.853134915
Average loss at step 4000: 52.7802186744
Average loss at step 6000: 33.4228466823
Average loss at step 8000: 23.2594624711
Average loss at step 10000: 17.8283199575
Nearest to from:  of,

## Step 6: Play & Visualize the embeddings

In [17]:
final_embeddings.shape

(50000, 128)

In [27]:
from numpy import linalg as LA

def word_vector(word):
    try:
        vector = final_embeddings[dictionary[word]]
        return vector
    except KeyError:
        return np.zeros(128)

def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (LA.norm(vec1) * LA.norm(vec2))

In [19]:
def plot_with_labels(low_dim_embs, labels, filename='tsne.png'):
    assert low_dim_embs.shape[0] >= len(labels), "More labels than embeddings"
    plt.figure(figsize=(18, 18))  # in inches
    for i, label in enumerate(labels):
        x, y = low_dim_embs[i, :]
        plt.scatter(x, y)
        plt.annotate(label,
                     xy=(x, y),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
    plt.savefig(filename)

In [20]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
    
tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000)
plot_only = 500
low_dim_embs = tsne.fit_transform(final_embeddings[:plot_only, :])
labels = [reverse_dictionary[i] for i in xrange(plot_only)]
plot_with_labels(low_dim_embs, labels)